In [96]:
# большая ячейка с импортами
import random
import os
import string

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report


import requests as rq
from bs4 import BeautifulSoup

import telebot 
from telebot import types 

import random
import json
import re
from datetime import datetime
import calendar

# !pip install natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)

#!pip install transliterate
from transliterate import translit

#!pip install markovify
import markovify

#!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [97]:
# api сайта https://openweathermap.org/, который позволяет получать погоду на ближайшие несколько дней
api_id = 'c4f85f9f56687c24d9bc1396dd40f79a'

In [98]:
# Данные
# Фамилии значенитых авторов
with open(r'authors_surnames.txt') as file:
    authors_surnames = file.read()
authors_surnames = authors_surnames[0:-2].split(' ')

# Цитаты
with open(r'quotes_reading.txt') as file:
    quotes = file.read()
quotes = quotes[0:-1].split('\n')

In [99]:
# Информация про время - сегодня, завтра и тд

# Список месяцев - они в усеченной форме, чтобы не лемматизировать входящий текст
months = ['0', 'январ', 'феврал', 'март', 'апрел', 'мая', 'июн', 'июл', 'август', 'сентябр', 'октябр', 'ноябр', 'декабр']

today = datetime.today()
today = today.replace(hour=0, minute = 0, second=0, microsecond =0)

# Удобнее ориентироваться, когда вся дата в одном формате - приводим все к timestamp
# Это также позволяет избежать проблемы с датами в конце/начале месяца
today_timestamp = int(today.timestamp())
tomorrow_timestamp = int(today_timestamp) + 86400
today_plus_two_timestamp = int(tomorrow_timestamp) + 86400
today_plus_three_timestamp = int(today_plus_two_timestamp) + 86400
today_plus_four_timestamp = int(today_plus_three_timestamp) + 86400
today_plus_five_timestamp = int(today_plus_four_timestamp) + 86400

# Здесь год, месяц и день в удобоваримом виде - чтобы выдавать текстовый аутпут
current_year = datetime.fromtimestamp(today_timestamp).year
good_years = list(set([datetime.fromtimestamp(today_timestamp).year, datetime.fromtimestamp(tomorrow_timestamp).year,
            datetime.fromtimestamp(today_plus_two_timestamp).year, datetime.fromtimestamp(today_plus_three_timestamp).year, 
            datetime.fromtimestamp(today_plus_four_timestamp).year, datetime.fromtimestamp(today_plus_five_timestamp).year]))
current_month = datetime.fromtimestamp(today_timestamp).month
good_months = list(set([datetime.fromtimestamp(today_timestamp).month, datetime.fromtimestamp(tomorrow_timestamp).month,
            datetime.fromtimestamp(today_plus_two_timestamp).month, datetime.fromtimestamp(today_plus_three_timestamp).month, 
            datetime.fromtimestamp(today_plus_four_timestamp).month, datetime.fromtimestamp(today_plus_five_timestamp).month]))
good_months_text = [months[i] for i in good_months]
current_day = datetime.fromtimestamp(today_timestamp).day
good_days = [datetime.fromtimestamp(today_timestamp).day, datetime.fromtimestamp(tomorrow_timestamp).day,
            datetime.fromtimestamp(today_plus_two_timestamp).day, datetime.fromtimestamp(today_plus_three_timestamp).day, 
            datetime.fromtimestamp(today_plus_four_timestamp).day, datetime.fromtimestamp(today_plus_five_timestamp).day]

current_week_days = [datetime.fromtimestamp(today_timestamp).weekday(), datetime.fromtimestamp(tomorrow_timestamp).weekday(),
                     datetime.fromtimestamp(today_plus_two_timestamp).weekday(), datetime.fromtimestamp(today_plus_three_timestamp).weekday(),
                    datetime.fromtimestamp(today_plus_four_timestamp).weekday(), datetime.fromtimestamp(today_plus_five_timestamp).weekday()]
wkdays_names = ['понедельник','вторник', 'сред', 'четверг', 'пятниц', 'суббот', 'воскресенье']
good_weekdays = {wkdays_names[day] : i for i, day in enumerate(current_week_days)}

## Интенты

In [100]:
# Бот распознает интенты приветствия, прощания, "узнать, как дела", благодарности, недовольства ботом, + литература и погода
with open(r'intents.json', encoding = 'utf-8') as file:
    data_str = file.read()

data_dict = json.loads(data_str)

phrases = []
classes_word = []
unique_classes = []

for intent in data_dict:
    for pattern in data_dict[intent]['patterns']:
        phrases.append(pattern)
        classes_word.append(intent)
    if intent not in unique_classes:
        unique_classes.append(intent)

indexed_classes =[]
for intent in classes_word:
    indexed_classes.append(unique_classes.index(intent))
    
vectorizer = CountVectorizer()
vectorizer.fit(phrases)
X_vec = vectorizer.transform(phrases)

X_train, X_test, y_train, y_test = train_test_split(X_vec,indexed_classes, test_size=0.25, random_state=42)

model = MLPClassifier()
model.fit(X_train, y_train)

C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier()

In [101]:
# Дополнительно: интенты да-нет

with open(r'my_intents.json', encoding = 'utf-8') as file:
    yes_no = file.read()

yes_no_dict = json.loads(yes_no)

yn_phrases = []
yn_classes_word = []
yn_unique_classes = []
yn_indexed_classes = []

for intent in yes_no_dict:
    for pattern in yes_no_dict[intent]['patterns']:
        yn_phrases.append(pattern)
        yn_classes_word.append(intent)
    if intent not in yn_unique_classes:
        yn_unique_classes.append(intent)

for intent in yn_classes_word:
    yn_indexed_classes.append(yn_unique_classes.index(intent))
    
yn_vectorizer = CountVectorizer()
yn_vectorizer.fit(yn_phrases)
yn_X_vec = yn_vectorizer.transform(yn_phrases)

yn_X_train, yn_X_test, yn_y_train, yn_y_test = train_test_split(yn_X_vec, yn_indexed_classes, test_size=0.25, random_state=42)

yn_model = MLPClassifier()
yn_model.fit(yn_X_train, yn_y_train)

yn_y_preds = yn_model.predict(yn_X_test)
# print(classification_report(yn_y_preds, yn_y_test))

C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Функции

In [102]:
# Выяснить, интент согласия или несогласия
def yes_or_no(text):
    text_vec = yn_vectorizer.transform([text])
    result = yn_model.predict(text_vec)[0]
    return yn_unique_classes[result]

In [103]:
# Найти город в тексте сообщения - с помощью names_extractor из natasha
def get_city(text):
    city = ''
    doc = Doc(text.title())
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        span.normalize(morph_vocab)
    for span in doc.spans:
        if span.type == 'LOC':
            span.extract_fact(names_extractor)
            city = span.normal
    if city != '':
        return city.title()
    return False

In [138]:
# Функция для выдачи информации по авторам
def authors_decorator(method):
    def output(self):
        if method(self) != False:
            return f'Тебе нравится {self.name}? Мне тоже! Здорово, что у нас хороший вкус!\n\Обычно тем, кто нравится этот автор, также нравятся следующие: {method(self)}'
        else:
            return f'Тебе нравится {self.name}? Мне тоже! Здорово, что у нас хороший вкус!'
    return output

In [105]:
# Функция-декоратор для получения координат
def coordinates_decorator(method):
    def get_coords(self,  *args):
        if method.__name__ == 'change_place':
            method(self,  *args)
        try:
            city_info = rq.get(f"http://api.openweathermap.org/geo/1.0/direct?q={self.place}&appid={api_id}")
            self.lat = city_info.json()[0]['lat']
            self.lon = city_info.json()[0]['lon']
        except:
            return False
        return method(self,  *args)
    return get_coords

## Классы

In [106]:
# Класс Город: атрибуты - название и координаты, методы - очистить/обнулить город, дать погоду на сейчас и на какую-то дату 
# (для них используются разные разделы openweathermap, поэтому удобнее разнести на 2 разных метода)

class City:
    
    def __init__(self, place):
        self.place = place
        self.lat = float
        self.lon = float
    
    def __str__(self):
        return f'Информация о городе {self.place} с координатами {self.lat}, {self.lon}'
    
    def __len__(self):
        return len(self.place)
        
    @coordinates_decorator
    def give_current_weather(self):
        current_weather = rq.get(f"https://api.openweathermap.org/data/2.5/weather?lat={self.lat}&lon={self.lon}&appid={api_id}&units=metric&lang=ru")
        description = current_weather.json()['weather'][0]['description']
        temp = current_weather.json()['main']['temp']
        feels = current_weather.json()['main']['feels_like']
        weather_is = f'{self.place}: {description}, температура {temp}°C (ощущается как {feels}°C)'
        return weather_is
    
    @coordinates_decorator
    def give_weather_for_date(self, time):
        future_weather = rq.get(f"http://api.openweathermap.org/data/2.5/forecast?lat={self.lat}&lon={self.lon}&appid={api_id}&units=metric&lang=ru")
        future_weather = future_weather.json()['list']
        # Даты выдаются в таймстемпах
        dates = [i['dt'] for i in future_weather]
        current_dates = [i for i, date in enumerate(dates) if time <= date < (time+86400)]
        
        if current_dates == []:
            return self.give_current_weather()

        answer = ''
        for i in current_dates:
            description = future_weather[i]['weather'][0]['description']
            temp = future_weather[i]['main']['temp']
            feels_like = future_weather[i]['main']['feels_like']
            when = datetime.fromtimestamp(future_weather[i]['dt']).time()
            answer += f'{when}: {description}, температура {temp}°C (ощущается как {feels_like}°C)\n'
            day = datetime.fromtimestamp(future_weather[i]['dt']).day
            month = datetime.fromtimestamp(future_weather[i]['dt']).month
        return f'Прогноз - {self.place} на {day}.{month}\n{answer}'
    
    def clear_city(self):
        self.place = ''
        self.lat = float
        self.lon = float
    
# Наследование: HomeCity/город, в котором пользователь живет. Используется как дефолтный, если не указывается другой город.
# Свой метод change_place для изменения города пользователя. Изменен метод clear_city - город не обнуляется, а возвращается
# к значению по умолчанию (мы живем в Москве -> город по умолчанию - Москва)

class HomeCity(City):
    
    def __init__(self, place='Москва'):
        super().__init__(place)
        self.lat = 55.7504461
        self.lon = 37.6174943
    
    @coordinates_decorator
    def change_place(self, new_place):
        self.place = new_place
        
    def clear_city(self):
        self.place = 'Москва'
        self.lat = 55.7504461
        self.lon = 37.6174943

In [107]:
# примеры с наследованием и переопределением операторов
m = HomeCity('Москва')
print(m)
print(len(m))
print(f'Координаты: {m.lat}, {m.lon}')
m.change_place('Токио')
print(f'Новые координаты: {m.lat}, {m.lon}')
m.clear_city()
print(f'Возврат к дефолтным координатам: {m.lat}, {m.lon}')
# Помогло заметить, что координаты не обнулялись после возврата к дефолтному городу - исправила

Информация о городе Москва с координатами 55.7504461, 37.6174943
6
Координаты: 55.7504461, 37.6174943
Новые координаты: 35.6828387, 139.7594549
Возврат к дефолтным координатам: 55.7504461, 37.6174943


In [108]:
# Класс дата
class Date:
    
    def __init__(self, message):
        self.text = message.lower()
        self.timestamp = 0
        
    def find_if_they_said_date(self):
    
        text_date = re.findall('сейчас|сегодня|завтра|послезавтра|через два дня|через три дня|через четыре дня|через 2 дня|через 3 дня|через 4 дня', self.text)
        day_month = re.findall('\d{1,2}\.\d{1,2}', self.text)
        day_month_text = re.findall('\d{1,2}?\s[январ|феврал|март|апрел|мая|июн|июл|август|сентябр|октябр|ноябр|декабр]+', self.text)
        text_weekday = re.findall('понедельник|вторник|сред|четверг|пятниц|суббот|воскресенье', self.text)

        if text_date != []:
            if text_date[0] != 'сейчас':
                self.timestamp = self.convert_text_date(text_date[0])


        elif text_weekday != [] and text_weekday[0] in good_weekdays.keys():
            weekday = good_weekdays[text_weekday[0]]
            a = good_days[weekday]
            b = current_month
            self.timestamp = self.convert_date(a, b)

        elif day_month != []:
            for i in day_month:
                a, b = i.split('.')
                if int(b) in good_months and int(a) in good_days:
                    self.timestamp = self.convert_date(a, b)
            if self.timestamp == 0:
                return False

        elif day_month_text != []:
            for i in day_month_text:
                a, b = i.split(' ')
                for i in good_months_text:
                    if b[0:2] == i[0:2] and int(a) in good_days:
                        b = months.index(i)
                        self.timestamp = self.convert_date(a, b)
                if self.timestamp == 0:
                    return False

        else:
            return False
        
    def convert_date(self, a, b):
        date = int(datetime.strptime(f'{a}.{b}.{current_year}','%d.%m.%Y').timestamp())
        return date

    # конвертирует дату, данную текстом, в таймстемп
    def convert_text_date(self,time):
        if time == 'сегодня':
            return today_timestamp
        elif time == 'завтра':
            return tomorrow_timestamp
        elif time == 'послезавтра' or time == 'через два дня' or time == 'через 2 дня':
            return today_plus_two_timestamp
        elif time == 'через три дня' or time == 'через 3 дня':
            return today_plus_three_timestamp
        else:
            return today_plus_four_timestamp
    
    def clear_date(self):
        self.text = ''
        self.timestamp = 0

In [109]:
# Класс для парсинга книжных сайтов, атрибуты: запрос пользователя, url, page и soup из beautifulsoup
# Методы: поиск книг на сайте, выдача случайной книги - модно использовать саму по себе, также используется в случае неудачи
# при поиске книг на сайте.
class getBooks:
    def __init__(self, book_query):
        self.query = book_query
        self.url = f"https://www.labirint.ru/search/{self.query}/?stype=0"
        self.book_info = ''
        self.page = rq.get(self.url)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
    
    def __str__(self):
        return f'Поиск книги {self.query} на сайте {self.url}'
    
    def give_random_book(self):
        bookish = ['Евгений Онегин (А.С.Пушкин)', 'Анна Каренина (Л.Н.Толстой)', 'Собачье сердце (М.А.Булгаков)', 'Гранатовый браслет (А.И. Куприн)', 'Дворянское гнездо (И.С. Тургенев)']
        return random.choice(bookish)
        
    def browse_books(self):
        try:
            books = self.soup.findAll("div", class_ = "product-cover")
            # Ищем первые три
            for i in books[:3]:
                book_number = i.find("a").get('href')
                book_name = i.find("a").get("title")
                book_price = i.find("span", class_ = "price-val").text
                book_price = re.sub('\t', '', book_price)
                book_price = re.sub("\n", '', book_price)
                book_price = re.sub('\r', '', book_price)
                self.book_info += f"- {book_name} ({book_price}): https://www.labirint.ru{book_number}\n"
            return self.book_info
        except:
            return f'Я не могу найти такой книги, но могу посоветовать другую - {give_random_book()}'

# Унаследованный класс для парсинга книг в книжном "Москва"
class getBooksMsk(getBooks):
    def __init__(self, book_query):
        super().__init__(book_query)
        self.url = f'https://www.moscowbooks.ru/search/?r46_search_query={self.query}&r46_input_query={self.query}'
    
    def browse_books(self):
        try:
            books = self.soup.findAll("div", class_ = "book-preview__content")
            assert books != []
            for i in books[:3]:
                book_number = i.find("a").get('href')
                book_name = i.find("div", class_ = "book_title")
                book_price = i.find("div", class_ = "price_tag").text
                self.book_info += f"- {book_name} ({book_price}): https://www.moscowbooks.ru/{book_number}\n"
            return self.book_info
        except:
            return f'Я не могу найти такой книги, но могу посоветовать другую - {self.give_random_book()}'

In [141]:
# Класс с информацией про автора книги
class Author:
    
    def __init__(self, sentence):
        self.sentence_with_name = sentence
        self.name = ''
        self.name_translit = ''
        self.similar = ''
        
    def check_if_author(self):
        msg_split = self.sentence_with_name.lower().split(' ')
        for i in msg_split:
            lemma = morph.parse(i)[0].normal_form
            if lemma in authors_surnames:
                self.name = lemma.title()
                return self.name
        return False

    def get_name(self):
        doc = Doc(self.sentence_with_name.title())
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        doc.parse_syntax(syntax_parser)
        doc.tag_ner(ner_tagger)
        for span in doc.spans:
            span.normalize(morph_vocab)
        for span in doc.spans:
            if span.type == 'PER':
                span.extract_fact(names_extractor)
                self.name = span.normal
        if self.name != '':
            return self.name
        else:
            return self.check_if_author()
        
    def transliterate_name(self):
        self.name_translit = translit(self.name, 'ru', reversed = True)
    
    @authors_decorator
    def similar_authors(self):
        text = re.sub('\s', '+', self.name)
        url = f"https://www.literature-map.com/{text}"
        page = rq.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        try:
            soup.findAll("div", {'id':'gnodMap'})[0].text
            a = soup.findAll("div", {'id':'gnodMap'})[0].text
            authors = a.split('\n')[2:7]
            authors_out = str(authors[0])
            for i in authors[1:]:
                authors_out += f', {i}'
            return authors_out

        except:
            self.transliterate_name
            text = self.name_translit
            try:
                soup.findAll("div", {'id':'gnodMap'})[0].text
                a = soup.findAll("div", {'id':'gnodMap'})[0].text
                authors = a.split('\n')[2:7]
                authors_out = str(authors[0])
                for i in authors[1:]:
                    authors_out += f', {i}'
                return authors_out
            except:
                return False

## Бот

In [144]:
bot = telebot.TeleBot('5973691708:AAHafjUNwDv-9wcV28P9Gz3826pAz2OUcSY')
api_id = 'c4f85f9f56687c24d9bc1396dd40f79a'
home_city = HomeCity()
city = City('')
date = Date('')

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Привет, я телеграм-бот! Пока я умею говорить только про погоду и немного про книги, но я быстро учусь. Давай общаться!")
    bot.send_message(message.from_user.id, 'В каком городе ты живешь?')
    bot.register_next_step_handler(message, change_city)

@bot.message_handler(commands=['change_city'])
def ask_about_home_city(message):
    bot.reply_to(message, "В каком городе ты живешь?")
    bot.register_next_step_handler(message, change_city)

def change_city(message):
    city = message.text
    if get_city(city) == False:
        bot.send_message(message.from_user.id, 'Что-то не могу найти этот город - попробуй еще раз!')
        bot.register_next_step_handler(message, change_city)
    else:
        home_city.change_place(get_city(city))
        bot.send_message(message.from_user.id, 'Здорово! Теперь можем поговорить о погоде или о  книгах.')
        
@bot.message_handler(func=lambda m: True)
def get_intent(message):
    text = message.text
    if get_city(text) != False:
        get_weather_info_from_message(message)
        
    else:    
        text_vec = vectorizer.transform([message.text])
        result = model.predict(text_vec)[0]
        intent = unique_classes[result]

        if intent == 'weather':
            get_weather_info_from_message(message)
        elif intent == 'literature':
            lit_choice(message)
        elif intent == 'goodbye':
            bot.send_message(message.from_user.id, text=random.choice(data_dict[intent]['responses']))
        else:
            bot.send_message(message.from_user.id, text=random.choice(data_dict[intent]['responses']))
            bot.send_message(message.from_user.id, 'Хочешь поговорить про погоду или книги?')

def get_weather_info_from_message(message):
    global city
    global date
    text = message.text
    date = Date(message.text)
    
    if get_city(text) != False and date.find_if_they_said_date() != False:
        city.place = get_city(text)
        date.find_if_they_said_date()
        if date.timestamp == 0:
            bot.send_message(message.from_user.id, city.give_current_weather())
        else:
            bot.send_message(message.from_user.id, city.give_weather_for_date(date.timestamp))
        city.clear_city()
        date.clear_date()
        bot.send_message(message.from_user.id, 'Что-то еще?')
        
    elif get_city(text) != False:
        city.place = get_city(text)
        bot.send_message(message.from_user.id, "Хочешь узнать, какая погода в этом городе сейчас?")
        bot.register_next_step_handler(message, get_yn_intent_when)
    
    elif date.find_if_they_said_date() != False:
        date.find_if_they_said_date()
        bot.send_message(message.from_user.id, f"Ты хочешь узнать, какая погода в городе {home_city.place}?")
        bot.register_next_step_handler(message, get_yn_intent_where)
        
    else:
        bot.send_message(message.from_user.id, f"Хочешь узнать, какая сейчас погода в городе {home_city.place}?")
        bot.register_next_step_handler(message, get_yn_intent_all)

def get_yn_intent_when(message):
    global city
    intent = yes_or_no(message.text)
    if intent == 'no':
        bot.send_message(message.from_user.id, 'Я могу сказать, какая погода сейчас или какая погода будет в ближайшие четыре дня. Какой день тебя интересует?')
        bot.register_next_step_handler(message, register_when)
    else:
        bot.send_message(message.from_user.id, city.give_current_weather())
        city.clear_city()
        bot.send_message(message.from_user.id, 'Что-то еще?')
    
    
def register_when(message):
    global city, date
    date = Date(message.text)

    if date.find_if_they_said_date() == False:
        bot.send_message(message.from_user.id, 'Пока я предсказываю погоду только на ближайшие дни. Какой день тебя интересует?')
        date.clear_date()
        bot.register_next_step_handler(message, register_when)
        
    else:
        date.find_if_they_said_date()
        if date.timestamp == 0:
            bot.send_message(message.from_user.id, city.give_current_weather())
        else:
            bot.send_message(message.from_user.id, city.give_weather_for_date(date.timestamp))
            
        city.clear_city()
        date.clear_date()
        bot.send_message(message.from_user.id, 'Что-то еще?')

def get_yn_intent_where(message):
    global date
    intent = yes_or_no(message.text)
    if intent == 'no':
        bot.send_message(message.from_user.id, 'Какой город тебя интересует?')
        bot.register_next_step_handler(message, register_where)
    else:
        if date.timestamp == 0:
            bot.send_message(message.from_user.id, home_city.give_current_weather())
        else:
            bot.send_message(message.from_user.id, home_city.give_weather_for_date(date.timestamp))
            date.clear_date()             
        bot.send_message(message.from_user.id, 'Что-то еще?')
                         
def register_where(message):
    global city, date
    cit = message.text
    if get_city(cit) == False:
        bot.send_message(message.from_user.id, "Кажется, я такого не знаю - попробуй другой!")
        bot.register_next_step_handler(message, register_where)
    else:
        city = City(get_city(cit))
        if date == "сейчас":
            bot.send_message(message.from_user.id, city.give_current_weather())
        else:
            bot.send_message(message.from_user.id, city.give_weather_for_date(date.timestamp))

        city.clear_city()
        date.clear_date()
        bot.send_message(message.from_user.id, 'Что-то еще?')
                         
def get_yn_intent_all(message):
    intent = yes_or_no(message.text)
    if intent == 'no':
        bot.send_message(message.from_user.id, 'Какой город и день тебя интересует? Я могу сказать, какая погода сейчас или какая погода будет в ближайшие четыре дня.')
        bot.register_next_step_handler(message, get_weather_info_from_message)
    else:
        bot.send_message(message.from_user.id, home_city.give_current_weather()) 
        bot.send_message(message.from_user.id, 'Что-то еще?')

# Книги
def book_shop(message):
    book = random.choice([getBooks(message.text), getBooksMsk(message.text)])
    bot.send_message(message.from_user.id, book.browse_books())
    bot.send_message(message.from_user.id, 'О чём еще хочешь поговорить?')
                         
def lit_choice(message):
    
    what_next = random.choice(['ask_about_author', 'random_quote', 'suggest_to_buy'])
    
    if what_next == 'suggest_to_buy':
        bot.send_message(message.from_user.id, 'Я очень люблю книги и могу подсказать тебе, где их купить. Тебе это интересно?')
        bot.register_next_step_handler(message, get_yn_intent_about_buying)

    elif what_next == "random_quote":
        gentext=markovify.Text(quotes, state_size =1, well_formed=True)
        bot.send_message(message.from_user.id, gentext.make_short_sentence(80))
        bot.send_message(message.from_user.id, 'О чём еще хочешь поговорить?')

    else:
        bot.send_message(message.from_user.id, 'Мои любимые авторы - Джонатан Франзен и Ричард Руссо. А какой твой любимый автор?')
        bot.register_next_step_handler(message, find_author)

def find_author(message):
    author = Author(message.text)
    if author.get_name() == False:
        bot.send_message(message.from_user.id, 'Кажется, я такого не знаю - попробуй другого автора!')
    else:
        author.get_name()
        bot.send_message(message.from_user.id, author.similar_authors())
        
    bot.send_message(message.from_user.id, 'О чём еще хочешь поговорить?')

def get_yn_intent_about_buying(message):
    text = message.text
    text_vec = yn_vectorizer.transform([text])
    result = yn_model.predict(text_vec)[0]
    intent = yn_unique_classes[result]

    if intent == 'yes':
        bot.send_message(message.from_user.id, 'Просто напиши мне, какую книгу ты хочешь купить!')
        bot.register_next_step_handler(message, book_shop)
    else:
        bot.send_message(message.from_user.id, 'Тогда давай поговорим про что-нибудь другое. Погода, книги?')
            
bot.polling(none_stop=True, timeout=123)

first place of struggle
we struggle here
we got here
or here?
are we here?
we got here


## Unit тесты

In [71]:
import unittest

In [94]:
class TestNotebook(unittest.TestCase):

    def testDates(self):
        print('Протестируем распознавание даты')
        # дата точно в прошлом
        date = Date('25.12')
        date.find_if_they_said_date()
        self.assertEqual(date.find_if_they_said_date(), False)
        
        # дата, которая возможно будет в прошлом
        date = Date('СКОЛЬКО градусов в Питере 27.12')
        date.find_if_they_said_date()
        if today_timestamp <= 1672088400:
            self.assertEqual(date.timestamp(), 1672088400)
        else:
            self.assertEqual(date.find_if_they_said_date(), False)
            
        # дата в будущем:
        date = Date('будет ли холодно 31 декабря?')
        date.find_if_they_said_date()
        if today_timestamp <= 1672434000 <= today_plus_five_timestamp:
            self.assertEqual(date.timestamp, 1672434000)
        else:
            self.assertEqual(find_if_they_said_date(), False)
        
        # дата слишком сильно в будущем:
        date = Date('погода в екб 01.02')
        date.find_if_they_said_date()
        if today_timestamp <= 1643662800 <= today_plus_five_timestamp:
            self.assertEqual(date.timestamp, 1672434000)
        else:
            self.assertEqual(date.find_if_they_said_date(), False)
            
        # даты в текстовом формате
        date = Date('погода в екб сегодня')
        date.find_if_they_said_date()
        self.assertEqual(date.timestamp, today_timestamp)
        
        date = Date('Какая погода через 2 дня')
        date.find_if_they_said_date()
        self.assertEqual(date.timestamp, today_plus_two_timestamp)
        
        date = Date('погода через 6 дня')
        date.find_if_they_said_date()
        self.assertEqual(date.find_if_they_said_date(), False)
        
    def testPlace(self):
        print('Тест распознавания города')
        self.assertEqual(get_city('Погода в Москве'), 'Москва')
        self.assertEqual(get_city('Сколько градусов в Красноярске'), 'Красноярск')
        self.assertEqual(get_city('какая погода'), False)
        self.assertEqual(get_city('будет ли жарко завтра в Оттаве'), 'Оттава')
        self.assertEqual(get_city('РАССКАЖИ О ПОГОДЕ В ВОЛГОГРАДЕ'), 'Волгоград')
        self.assertEqual(get_city('холодно в нижнем тагиле'), 'Нижний Тагил')
        self.assertEqual(get_city('кАкАя ПоГоДа в АстАнЕ'), 'Астана')

In [95]:
unittest.main(argv=[''], verbosity=2, exit=False)

testDates (__main__.TestNotebook) ... ok
testPlace (__main__.TestNotebook) ... 

Протестируем распознавание даты
Тест распознавания города


C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\slovnet\exec\model.py:419: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  zero = np.zeros((batch_size, 1), dtype=np.long)
ok

----------------------------------------------------------------------
Ran 2 tests in 0.040s

OK
